In [16]:
%run utils/devtools.py

In [17]:
%reloadmypkg utils

import pandas as pd
from utils.url2platform import *
from utils.standardise_url import *
from utils.fetch_info import *
from utils.dump_df_to_json import *

✅ Reloaded package 'utils' and its submodules.


In [18]:
sheet_name = "../raw-data/leetcode/75-response-data.json"
export_file = "../cleaned-data/leetcode/75.json"

In [19]:
with open(sheet_name) as file:
    raw = json.load(file)

flattened_data = [
    "https://leetcode.com/problems/" + question["titleSlug"]
    for subgroup in raw["data"]["studyPlanV2Detail"]["planSubGroups"]
    for question in subgroup["questions"]
]

df = pd.DataFrame(flattened_data, columns=['url'])

In [20]:
df.head(2)

,url
0,https://leetcode.com/problems/merge-strings-al...
1,https://leetcode.com/problems/greatest-common-...


In [21]:
df.shape

(75, 1)

In [22]:
# Find the duplicates

dupes_df = df[df.duplicated(subset="url", keep=False)]
dupes_df.shape

(0, 1)

In [23]:
import nest_asyncio
nest_asyncio.apply()

standardised_df = await standardise_urls_async(df, "leetcode-75-urls-expanded.json")

Filter list is empty. Trying to expand all 75 URLs.

Processing batch 1/1
Loaded 0 results from checkpoint
Processing 75 pending URLs in this batch


Expanding URLs: 100%|██████████| 75/75 [00:01<00:00, 38.97it/s]

Progress saved to leetcode-75-urls-expanded.json
----------------------------------------------------------------------
Progress saved to leetcode-75-urls-expanded.json
----------------------------------------------------------------------

75/75 URLs expanded in 3.10 seconds.


In [24]:
standardised_df.shape

(75, 3)

In [25]:
# Adding type column
standardised_df['type'] = standardised_df['expanded_stripped_url'].apply(
    lambda url: "problem" if is_problem_url(url) else "tutorial"
)

In [26]:
# Adding id_base, id, title, platform columns
import nest_asyncio
nest_asyncio.apply()

final_df = await fetch_info(standardised_df, "leetcode-75-info-added.json", resume=True)

Found 75 URLs.

Batch 1/1: items 0-74
Loaded 0 from checkpoint
Processing 75 URLs...


Fetching items: 100%|██████████| 75/75 [00:04<00:00, 17.83it/s]

Saved progress (75 items) to leetcode-75-info-added.json
------------------------------
Saved progress (75 items) to leetcode-75-info-added.json
------------------------------

Fetched 75/75 items in 4.2s.


In [27]:
final_df.shape

(75, 8)

In [28]:
dump_df_to_json(final_df, export_file)